###**DDoS Attack Prediection by** 
###**Random Forest Decision Tree, Gradient Boosted Tree Classifier, Logistic Regression and Linear SVC**


In [1]:
%pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import *
from pyspark.ml import *
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.mllib.evaluation import *
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit
from pyspark.context import *
from pyspark.sql.session import SparkSession

In [2]:
%pyspark
PYSPARK_CLI = False
if PYSPARK_CLI:
	sc = SparkContext.getOrCreate()
	spark = SparkSession(sc)

####**To define the required functions**
##### The function *get_feature_imp* is used in the Feature importance calculation part.
##### The function *binary_metric* is used the metric calculation part.


In [4]:
%pyspark
#Note: Required Function
def get_feature_imp(col,imp):
    tmp_ft_data_v2=[]
    tmp_c=0
    tmp_ft_data_v1 = list(zip(col,imp))
    for c1,c2 in tmp_ft_data_v1:
        tmp_ft_data_v2.insert(tmp_c,[c1,float(c2)])
        tmp_c=tmp_c+1
    tmp_df = spark.createDataFrame(data=tmp_ft_data_v2,schema=['Column Name','Features'])
    return tmp_df
    
    

def binary_metric(name,predicted):
    evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
    auc = evaluator.evaluate(predicted)
    tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
    fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
    tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
    fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
    #AUC,Recall,Precision,tp,fp,tn,fn
    return [name,auc, tp / (tp + fn),tp / (tp + fp),tp,fp,tn,fn]
    
    

####**To Read the dataset in csv format from HDFS and To Infer the Schema**


In [6]:
%pyspark
# File location and type
file_location = "/user/vmehala/project/5560/"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
org_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [7]:
%pyspark
org_df.printSchema()

####**To get information about the dataframe (df)**


In [9]:
%pyspark
print("Total No. of Columns : ",len(org_df.columns))
print("List of Available Colunms :\n",org_df.columns)


####**To clean the dataframe**

The columns of the Source IPs , the Destination IPs and the Flow IDs have around 4500 distinct values and do not yeild any specific pattern for prediction.
The columns that are not used as features are dropped.

In [11]:
%pyspark
df_clean = org_df.drop("_c0", "Dst IP", "Flow Byts/s", "Flow ID", "Flow Pkts/s", "Src IP", "Timestamp")

In [12]:
%pyspark
df_clean.groupBy("Label").count().show()

 
#### **To sample the dataframe based on equal fraction of values in the Label column**


In [14]:
%pyspark
v1_seed=5561
v2_seed=5562
sample_dateset_1 = df_clean.sampleBy(col("Label"), fractions={"Benign": 0.007,"ddos": 0.007}, seed=v1_seed)

In [15]:
%pyspark
sample_dateset_1.groupBy("Label").count().show()

####**To select the required columns for train and test datasets**
The datatype of the Label column is changed from string to double and the value of "ddos" is changed to "1.0" and the value of "Benign" is changed to "0.0"

In [17]:
%pyspark
#ML_Traning_Dataset = sample_dateset_1.select("Src Port","Dst Port","Protocol","Flow Duration","Tot Fwd Pkts","Tot Bwd Pkts","TotLen Fwd Pkts","TotLen Bwd Pkts","Fwd Pkt Len Max","Fwd Pkt Len Min","Fwd Pkt Len Mean","Fwd Pkt Len Std","Bwd Pkt Len Max","Bwd Pkt Len Min","Bwd Pkt Len Mean","Bwd Pkt Len Std","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Tot","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max","Fwd IAT Min","Bwd IAT Tot","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags","Fwd Header Len","Bwd Header Len","Fwd Pkts/s","Bwd Pkts/s","Pkt Len Min","Pkt Len Max","Pkt Len Mean","Pkt Len Std","Pkt Len Var","FIN Flag Cnt","SYN Flag Cnt","RST Flag Cnt","PSH Flag Cnt","ACK Flag Cnt","URG Flag Cnt","CWE Flag Count","ECE Flag Cnt","Down/Up Ratio","Pkt Size Avg","Fwd Seg Size Avg","Bwd Seg Size Avg","Fwd Byts/b Avg","Fwd Pkts/b Avg","Fwd Blk Rate Avg","Bwd Byts/b Avg","Bwd Pkts/b Avg","Bwd Blk Rate Avg","Subflow Fwd Pkts","Subflow Fwd Byts","Subflow Bwd Pkts","Subflow Bwd Byts","Init Fwd Win Byts","Init Bwd Win Byts","Fwd Act Data Pkts","Fwd Seg Size Min","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min",(col("Label") == "ddos").cast("Double").alias("label"))
ML_Traning_Dataset = df_clean.select("Src Port","Dst Port","Protocol","Flow Duration","Tot Fwd Pkts","Tot Bwd Pkts","TotLen Fwd Pkts","TotLen Bwd Pkts","Fwd Pkt Len Max","Fwd Pkt Len Min","Fwd Pkt Len Mean","Fwd Pkt Len Std","Bwd Pkt Len Max","Bwd Pkt Len Min","Bwd Pkt Len Mean","Bwd Pkt Len Std","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Tot","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max","Fwd IAT Min","Bwd IAT Tot","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags","Fwd Header Len","Bwd Header Len","Fwd Pkts/s","Bwd Pkts/s","Pkt Len Min","Pkt Len Max","Pkt Len Mean","Pkt Len Std","Pkt Len Var","FIN Flag Cnt","SYN Flag Cnt","RST Flag Cnt","PSH Flag Cnt","ACK Flag Cnt","URG Flag Cnt","CWE Flag Count","ECE Flag Cnt","Down/Up Ratio","Pkt Size Avg","Fwd Seg Size Avg","Bwd Seg Size Avg","Fwd Byts/b Avg","Fwd Pkts/b Avg","Fwd Blk Rate Avg","Bwd Byts/b Avg","Bwd Pkts/b Avg","Bwd Blk Rate Avg","Subflow Fwd Pkts","Subflow Fwd Byts","Subflow Bwd Pkts","Subflow Bwd Byts","Init Fwd Win Byts","Init Bwd Win Byts","Fwd Act Data Pkts","Fwd Seg Size Min","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min",(col("Label") == "ddos").cast("Double").alias("label"))


####**To split the dataframe between train and test dataframes with the ratio of 70% and 30%, repectively.**


In [19]:
%pyspark
splits = ML_Traning_Dataset.randomSplit([0.7, 0.3])
train_V1 = splits[0]
test_V1 = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train_V1.count()
test_rows = test_V1.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

In [20]:
%pyspark
featuresList= train_V1.columns
print("list of features : \n ",featuresList)


####**To define the required lists for merging and comparing the results among models using diffrent algorithms**
- master pipeline is used to create a list of all the models of the algorithms and features pipelines.
- master_models is used to create a list of TrainValidationSplit evaluators of all the models.
- master_prediction is used to create a list of all the trasformations of all the evaluated models.



In [22]:
%pyspark
master_pipeline=[]
master_models=[]
master_preduction=[]
master_model_result=[]

#####**A VectorAssembler that combines categorical features into a single vector**
#####**A VectorIndexer that creates indexes for a vector of categorical features**
#####**A VectorAssembler that creates a vector of continuous numeric features**
#####**A MinMaxScaler that normalizes continuous numeric features**



In [24]:
%pyspark

#strIdx1 = StringIndexer(inputCol = "Src IP", outputCol = "SrcIPIdx")
#strIdx2 = StringIndexer(inputCol = "Dst IP", outputCol = "DstIPIdx")


catVect = VectorAssembler(inputCols = ["Src Port","Dst Port","Protocol","Fwd PSH Flags", "Bwd PSH Flags", "Fwd URG Flags", "Bwd URG Flags", "Fwd Header Len", "Bwd Header Len"],handleInvalid="skip", outputCol="catFeatures")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")

numVect = VectorAssembler(inputCols = ["Flow Duration", "Tot Fwd Pkts", "Tot Bwd Pkts", "TotLen Fwd Pkts", "TotLen Bwd Pkts", "Fwd Pkt Len Max", "Fwd Pkt Len Min", "Fwd Pkt Len Mean", "Fwd Pkt Len Std", "Bwd Pkt Len Max", "Bwd Pkt Len Min", "Bwd Pkt Len Mean", "Bwd Pkt Len Std", "Flow IAT Mean", "Flow IAT Std", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Tot", "Fwd IAT Mean", "Fwd IAT Std", "Fwd IAT Max", "Fwd IAT Min", "Bwd IAT Tot", "Bwd IAT Mean", "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min", "Fwd Pkts/s", "Bwd Pkts/s", "Pkt Len Min", "Pkt Len Max", "Pkt Len Mean", "Pkt Len Std", "Pkt Len Var", "FIN Flag Cnt", "SYN Flag Cnt", "RST Flag Cnt", "PSH Flag Cnt", "ACK Flag Cnt", "URG Flag Cnt", "CWE Flag Count", "ECE Flag Cnt", "Down/Up Ratio", "Pkt Size Avg", "Fwd Seg Size Avg", "Bwd Seg Size Avg", "Fwd Byts/b Avg", "Fwd Pkts/b Avg", "Fwd Blk Rate Avg", "Bwd Byts/b Avg", "Bwd Pkts/b Avg", "Bwd Blk Rate Avg", "Subflow Fwd Pkts", "Subflow Fwd Byts", "Subflow Bwd Pkts", "Subflow Bwd Byts", "Init Fwd Win Byts", "Init Bwd Win Byts", "Fwd Act Data Pkts", "Fwd Seg Size Min", "Active Mean", "Active Std", "Active Max", "Active Min", "Idle Mean", "Idle Std", "Idle Max", "Idle Min"], handleInvalid="skip",outputCol="numFeatures")

minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")





####**To tune the hyper parameters and to pipeline the algorithms and features**
Hyper parameter tuning is performed either during the related algorithm function call or by calling ParamGridBuilder() function. In the latter case, multiple parameters can be tested. 




In [26]:
%pyspark
featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")
#All the model
gbtc = GBTClassifier(labelCol="label", featuresCol="features",maxBins=500, featureSubsetStrategy="auto")
rfc = RandomForestClassifier(labelCol="label", featuresCol="features",maxBins=520, numTrees=20)
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
lsvc = LinearSVC(labelCol="label")
#Master Pipeline
master_pipeline=[]
master_pipeline.insert(0,["GBTClassifier",Pipeline(stages=[catVect, catIdx, numVect, minMax, featVect, gbtc])])
master_pipeline.insert(1,["RandomForestClassifier",Pipeline(stages=[catVect, catIdx, numVect, minMax, featVect, rfc])])
master_pipeline.insert(2,["LogisticRegression",Pipeline(stages=[catVect, catIdx, numVect, minMax, featVect, lr])])
master_pipeline.insert(3,["LinearSVC",Pipeline(stages=[catVect, catIdx, numVect, minMax, featVect, lsvc])])




In [27]:
%pyspark
master_pipeline

##### To try a range of parameters for hyper parameter tuning of Gradient Boosted Tree Classifier


In [29]:
%pyspark
GBT_TVS_paramGrid = ParamGridBuilder() \
  .addGrid(gbtc.maxDepth, [5, 10]) \
  .addGrid(gbtc.maxIter, [5, 10]) \
  .build()

####**To build the model by TrainValidationSplit and Gradient Boosted Classifier, and Train Ratio split of 80%**


In [31]:
%pyspark
GBT_TVS = TrainValidationSplit(estimator=master_pipeline[0][1], evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=GBT_TVS_paramGrid, trainRatio=0.8)
master_models.insert(0,["GBT TVS",GBT_TVS.fit(train_V1)])
print("GBT model complete")

##### To try a range of parameters for hyper parameter tuning of Random Forest Classifier


In [33]:
%pyspark
RF_TVS_paramGrid = ParamGridBuilder() \
  .addGrid(rfc.maxDepth, [10, 15]) \
  .addGrid(rfc.numTrees, [20]) \
  .build()


####**To build the model by TrainValidationSplit and Random Forest Classifier, and Train Ratio split of 80%**


In [35]:
%pyspark
RF_TVS = TrainValidationSplit(estimator=master_pipeline[1][1], evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=RF_TVS_paramGrid, trainRatio=0.8)
master_models.insert(1,["RF TVS",RF_TVS.fit(train_V1)])
print("RF model complete")


####**To build the model by TrainValidationSplit and Logistic Regression, and Train Ratio split of 80%**



In [37]:
%pyspark

LR_TVS_paramGrid = ParamGridBuilder() \
  .addGrid(lr.regParam, [0.3]) \
  .build()

In [38]:
%pyspark
LR_TVS = TrainValidationSplit(estimator=master_pipeline[2][1], evaluator=BinaryClassificationEvaluator(),estimatorParamMaps=LR_TVS_paramGrid, trainRatio=0.8)
master_models.insert(2,["LogisticRegression",LR_TVS.fit(train_V1)])
print("LR model complete")

In [39]:
%pyspark
master_models

####**To build the model by TrainValidationSplit and linear SVC, and Train Ratio split of 80%**


In [41]:
%pyspark
'''LSVC_TVS = TrainValidationSplit(estimator=master_pipeline[3][1], evaluator=BinaryClassificationEvaluator(), trainRatio=0.8)
master_models.insert(3,["LinearSVC",LSVC_TVS.fit(train_V1)])
print("LSVC model complete")'''

In [42]:
%pyspark
master_models.insert(3,["LinearSVC",master_pipeline[3][1].fit(train_V1)])

In [43]:
%pyspark
master_models

####**To transform the test dataframe**


In [45]:
%pyspark
tmp=0
master_preduction=[]
for name,tmp_model in master_models:
    master_preduction.insert(tmp,[name,tmp_model.transform(test_V1)])
    tmp=tmp+1


####**To calculate the evaluation metrics: Precision, Recall and AUC**


In [47]:
%pyspark
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
for name,tmp_preduction in master_preduction:
    auc = evaluator.evaluate(tmp_preduction)
    print(name," : AUC = ", auc)

In [48]:
%pyspark
print(binary_metric(master_preduction[0][0],master_preduction[0][1]))

In [49]:
%pyspark
columns_name = ['Model_name',"AUC","Recall","Precision","tp","fp","tn","fn"]
result_df = spark.createDataFrame(data=[binary_metric(master_preduction[0][0],master_preduction[0][1])],schema=columns_name)

In [50]:
%pyspark
for name,tmp_preduction  in master_preduction[1:]:
    result_df=result_df.union(spark.createDataFrame(data=[binary_metric(name,tmp_preduction)],schema=columns_name))
result_df.show()

In [51]:
%pyspark
z.show(result_df)

####**Feature Importance for Random Forest Classifier**


In [53]:
%pyspark
rfcModel= master_pipeline[1][1].fit(train_V1)
print("model complete")

In [54]:

%pyspark
rfcModel.stages[-1].featureImportances

In [55]:
%pyspark
ft_df_model1=get_feature_imp(featuresList,rfcModel.stages[-1].featureImportances)
ft_df_model2 = ft_df_model1.select("Column Name",round(col("Features"),4)).withColumnRenamed("round(Features, 4)", "Features Importance").sort(desc("Features Importance"))
ft_df_model2.show(80)


####**Feature Importance for Gradient Boosted Classifier**


In [57]:
%pyspark
gbtcModel= master_pipeline[0][1].fit(train_V1)
print("model complete")


In [58]:
%pyspark
gbtcModel.stages[-1].featureImportances


In [59]:
%pyspark
ft_df_model3=get_feature_imp(featuresList,gbtcModel.stages[-1].featureImportances)
ft_df_model4 = ft_df_model3.select("Column Name",round(col("Features"),4)).withColumnRenamed("round(Features, 4)", "Features Importance").sort(desc("Features Importance"))
ft_df_model4.show(80)


####**Feature Importance for logistic Regression**


In [61]:
%pyspark
lrModel= master_pipeline[2][1].fit(train_V1)
print("model complete")

In [62]:
%pyspark
lrModel.stages[-1].featureImportances



In [63]:
%pyspark
ft_df_model5=get_feature_imp(featuresList,lrModel.stages[-1].featureImportances)
ft_df_model6 = ft_df_model5.select("Column Name",round(col("Features"),4)).withColumnRenamed("round(Features, 4)", "Features Importance").sort(desc("Features Importance"))
ft_df_model6.show(80)
